# 🤖 ML Pipelines y Feature Stores

Objetivo: diseñar pipelines reproducibles de datos para ML, integrar feature stores (Feast/Tecton) y aplicar MLOps básico (versionado, reentrenamiento, monitoreo).

- Duración: 120–150 min
- Dificultad: Alta
- Prerrequisitos: Python ML básico, pipelines batch/streaming

## 1. Pipeline de datos para ML: ETL → Feature Engineering → Training → Serving

- Extract: fuentes raw (transacciones, logs, eventos).
- Transform: limpieza, agregaciones por ventana, joins.
- Feature Engineering: crear features (ratios, lags, embeddings).
- Training: dataset versionado, experimentos (MLflow/Weights&Biases).
- Serving: online (API real-time) y offline (batch predictions).

## 2. Feature Store: concepto y práctica con Feast (demo local)

In [ ]:
# Nota: instala feast si quieres ejecutar (no en requirements por defecto)
feast_demo = r'''
# feature_repo/feature_definitions.py
from feast import Entity, FeatureView, Field, FileSource
from feast.types import Float32, Int64
from datetime import timedelta

cliente = Entity(name='cliente_id', join_keys=['cliente_id'])

source = FileSource(
    path='data/features.parquet',
    timestamp_field='event_timestamp'
)

cliente_fv = FeatureView(
    name='cliente_features',
    entities=[cliente],
    ttl=timedelta(days=30),
    schema=[
        Field(name='total_compras', dtype=Float32),
        Field(name='num_transacciones', dtype=Int64),
    ],
    source=source
)
# feast apply
# feast materialize-incremental
# store.get_online_features(...)
'''
print(feast_demo.splitlines()[:25])

## 3. Versionado de datasets y experimentos con MLflow

In [ ]:
mlflow_snippet = r'''
import mlflow
mlflow.set_tracking_uri('http://localhost:5000')
mlflow.set_experiment('ventas_prediccion')

with mlflow.start_run():
    mlflow.log_param('model', 'xgboost')
    mlflow.log_param('n_estimators', 100)
    mlflow.log_metric('rmse', 0.85)
    mlflow.log_artifact('model.pkl')
'''
print(mlflow_snippet)

## 4. Reentrenamiento automatizado (Airflow + MLflow)

- DAG semanal: extraer nuevos datos, generar features, entrenar, evaluar.
- Si métrica mejora sobre baseline → registrar modelo, promover a producción.
- Si drift detectado → alerta y reentrenamiento fuera de calendario.

## 5. Serving online y batch

- Online: endpoint REST (FastAPI + modelo en memoria o via MLflow Model Registry).
- Batch: Spark job semanal para scoring de todo el catálogo/clientes.
- Caché de features online (Redis) para latencia < 50ms.

## 6. Monitoreo de modelos

- Data drift: distribución de features cambia (KS test, PSI).
- Concept drift: relación X→Y cambia (performance degrada).
- Métricas de negocio: precisión, recall, ROC-AUC, ingresos.
- Alertas automáticas y rollback si threshold cruzado.